# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [80]:
# import nltk and parsing dictionaries
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# import libraries and functions
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM CleanedMessages", engine)
X = df['message']
y = df.drop(columns=['message', 'id','original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [56]:
def tokenize(text):
    # delete punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text)
    
    # Tokenize 
    tokens = word_tokenize(text)
    
    # lemmatize
    lemmatizer = WordNetLemmatizer()


    # Remove stop words    
    cleaned_tokens = []
    for toke in tokens:
        clean_toke = lemmatizer.lemmatize(toke, pos='n').strip()
        clean_toke = lemmatizer.lemmatize(clean_toke, pos='v')
        cleaned_tokens.append(clean_toke)    
    
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [57]:
# Using RandomForrest Classifier for initial pass as it seesm a best fit for the problem type
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [63]:
# use fitted model to get predicted values
predicted = pipeline.predict(X_test)

# select columns from y to show f1 score
target_names = y.columns
print(classification_report(y_test, predicted, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      6002
               request       0.36      0.06      0.10      1308
                 offer       0.00      0.00      0.00        32
           aid_related       0.44      0.20      0.28      3203
          medical_help       0.02      0.00      0.00       611
      medical_products       0.04      0.01      0.01       375
     search_and_rescue       0.14      0.00      0.01       210
              security       0.00      0.00      0.00       161
              military       0.40      0.01      0.02       259
           child_alone       0.00      0.00      0.00         0
                 water       0.06      0.00      0.01       474
                  food       0.15      0.01      0.01       831
               shelter       0.10      0.01      0.01       707
              clothing       0.00      0.00      0.00       124
                 money       0.11      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [76]:
# parameters used for passing to the Pipeline. The parameter passed to the estimator:
# criterion: measures the quality of the split
# class_weight: attempts to balance imballanced data
parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"],
              'clf__estimator__class_weight': ['balanced', None]
             }

# Grid CV for testing
cv = GridSearchCV(pipeline, param_grid=parameters)

# This step is slow, it is running against a lot of data and running many different combinations
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l1', 'l2'], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [77]:
# best estimator, best score, grid_scores_ for the GridSearchCV results
print(f'best est: {cv.best_estimator_} \n')
print(f'best score: {cv.best_score_} \n')
print(f'scores: {cv.grid_scores_} \n')

best est: Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]) 

best score: 0.15569123419101613 

scores: [mean: 0.14702, std: 0.00288, params: {'clf__estimator__class_weight': 'balanced', 'clf__estimator__criterion': 'gini', 'tfidf__norm': 'l1'}, mean: 0.14691, std: 0.00319, params: {'clf__estimator__class_weight': 'balanced', 'clf__estimator__criterion': 'gini', 'tfidf__norm': 'l2'}, mean: 0.15008, std: 0.00352, params: {'clf__estimator__class_weight': 'balanced', 'clf__estimator__criterion': 'entropy', 'tfidf__norm': 'l1'}, mean: 0.14779, std: 0.00335, params: {'clf__estimator__class_weight': 'balanced', 'clf_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [78]:
cvpredicted = cv.predict(X_test)

# ideally, this should be a function since I have called it twice now
target_names = y.columns
print(classification_report(y_test, cvpredicted, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.94      0.84      6002
               request       0.41      0.07      0.12      1308
                 offer       0.00      0.00      0.00        32
           aid_related       0.44      0.21      0.28      3203
          medical_help       0.00      0.00      0.00       611
      medical_products       0.00      0.00      0.00       375
     search_and_rescue       0.00      0.00      0.00       210
              security       0.00      0.00      0.00       161
              military       0.33      0.00      0.01       259
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       474
                  food       0.17      0.00      0.01       831
               shelter       0.11      0.00      0.00       707
              clothing       0.00      0.00      0.00       124
                 money       0.33      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
## I will try other algorithms once I have the work flow for the project complete

### 9. Export your model as a pickle file

In [82]:
pickle.dump(cv, open('learningmodel.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.